In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from PIL import Image

from scribbles_creator import *
from cellpose_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define where the images are located and what data to consider

In [2]:
folder_path = "./cellpose_results/data/pred08"

In [3]:
mode = "all"
bins = [0.1, 1, 1.25]#[0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = ["x"] #all_suff[:5]
prediction_types = ["dino"] #["convpaint", "ilastik"] # ["convpaintCh1", "ilastikCh1"]

## Analyse the created scribbles and segmentations (perc. labelled vs. acc.) from saved files

In [4]:
df = pd.DataFrame(columns=['img_num', 'prediction type', 'scribbles mode', 'scribbles bin', 'suffix', 'class_1_pix_gt', 'class_2_pix_gt', 'pix_labelled', 'class_1_pix_labelled', 'class_2_pix_labelled', 'pix_in_img', 'perc. labelled', 'accuracy', 'image', 'ground truth', 'scribbles', 'prediction'])

for img_num in range(0, 540, 10):
    for bin in bins:
        suff = ["x"] if bin == 1.25 else ["a"]
        for s in suff:
            for pred in prediction_types:
                res = analyse_cellpose_single_file(folder_path, img_num, mode=mode, bin=bin, suff=s, pred_tag=pred, show_res=False)
                df = pd.concat([df, res], ignore_index=True)
df[:5]                

C:\Users\roman\AppData\Local\Temp\ipykernel_13404\694018349.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, res], ignore_index=True)


,img_num,prediction type,scribbles mode,scribbles bin,suffix,class_1_pix_gt,class_2_pix_gt,pix_labelled,class_1_pix_labelled,class_2_pix_labelled,pix_in_img,perc. labelled,accuracy,image,ground truth,scribbles,prediction
0,0,dino,all,0.10,a,101577,94519,174,84,90,196096,0.088732,0.704905,./cellpose_results/data/pred08/000_img.png,./cellpose_results/data/pred08/000_ground_trut...,./cellpose_results/data/pred08/000_scribbles_a...,./cellpose_results/data/pred08/000_dino_all_00...
1,0,dino,all,1.00,a,101577,94519,1932,997,935,196096,0.985232,0.738154,./cellpose_results/data/pred08/000_img.png,./cellpose_results/data/pred08/000_ground_trut...,./cellpose_results/data/pred08/000_scribbles_a...,./cellpose_results/data/pred08/000_dino_all_01...
2,0,dino,all,1.25,x,101577,94519,19188,10066,9122,196096,9.785003,0.747027,./cellpose_results/data/pred08/000_img.png,./cellpose_results/data/pred08/000_ground_trut...,./cellpose_results/data/pred08/000_scribbles_a...,./cellpose_results/data/pred08/000_dino_all_01...
3,10,dino,all,0.10,a,120609,75487,179,109,70,196096,0.091282,0.725543,./cellpose_results/data/pred08/010_img.png,./cellpose_results/data/pred08/010_ground_trut...,./cellpose_results/data/pred08/010_scribbles_a...,./cellpose_results/data/pred08/010_dino_all_00...
4,10,dino,all,1.00,a,120609,75487,1915,1179,736,196096,0.976562,0.784468,./cellpose_results/data/pred08/010_img.png,./cellpose_results/data/pred08/010_ground_trut...,./cellpose_results/data/pred08/010_scribbles_a...,./cellpose_results/data/pred08/010_dino_all_01...


Analyse the comparison of convpaint models

In [ ]:
folder_path = "results\data\pred06"
prediction_types = ["convpaint_l-0_s-1",
                    "convpaint_l-0_s-1-2",
                    "convpaint_l-0_s-1-2-4",
                    "convpaint_l-0_s-1-2-4-8",
                    "convpaint_l-0-2_s-1",
                    "convpaint_l-0-2_s-1-2",
                    "convpaint_l-0-2_s-1-2-4",
                    "convpaint_l-0-2_s-1-2-4-8"] # ["convpaintCh1", "ilastikCh1"]

df = pd.DataFrame(columns=['img_num', 'prediction type', 'scribbles mode', 'scribbles bin', 'suffix', 'class_1_pix_gt', 'class_2_pix_gt', 'pix_labelled', 'class_1_pix_labelled', 'class_2_pix_labelled', 'pix_in_img', 'perc. labelled', 'accuracy', 'image', 'ground truth', 'scribbles', 'prediction'])

for img_num in range(0, 540, 10):
    for pred in prediction_types:
        res = analyse_cellpose_single_file(folder_path, img_num, mode="all", bin=1.25, suff="x", pred_tag=pred, show_res=False)
        df = pd.concat([df, res], ignore_index=True)
df.shape

Save the data frame in a csv file:

In [5]:
from datetime import datetime
time_stamp = datetime.now().strftime("%y%m%d%H%M%S")
file_name = f"test_labels_vs_acc_{time_stamp}.csv"
df.to_csv(file_name, index=False)